In [ ]:
 #!pip install numpy pandas tqdm scikit-learn segmentation-models tensorflow-addons 

In [ ]:
#!pip install albumentations --user

In [1]:
import tensorflow as tf
print(tf.__version__)
print('GPU Available: ', tf.test.is_gpu_available())

2.10.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Available:  True


In [4]:
import tensorflow as tf
import os
import sys
os.environ['SM_FRAMEWORK'] = 'tf.keras'
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score

module_path = "F:\\Segmentacion_Automatica\\colab_glottis\\GlottisNetV2"
if module_path not in sys.path:
    sys.path.append(module_path)
#Drive
#from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Utils.DataGenerator import DataGenerator
#from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Utils.data import load_data, metric_mape, mape_ap, mape_pp
#from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Utils.Callbacks import get_callbacks
#from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Models.GlottisNetV2_e import glottisnetV2_e

from Models.GlottisNetV2_e import glottisnetV2_e
from Utils.DataGenerator_onlySeg import DataGenerator
from Utils.data import load_data, metric_mape, mape_ap, mape_pp
from Utils.Callbacks import get_callbacks

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [9]:
import pandas as pd
import os

from tqdm import tqdm

def is_image_valid(image_path):
    return os.path.getsize(image_path) > 0

# Asegúrate de que las rutas a las imágenes estén correctamente definidas
img_training = "F:/Segmentacion_Automatica/new_dataset" # TODO

N_train = 1999

cols = ['z', 'path']
df_imgs_train = pd.DataFrame(columns=cols)
df_segs_train = pd.DataFrame(columns=cols)

for i in tqdm(range(1, N_train )):  # Asegúrate de que el rango esté correctamente definido
    rgb_path = os.path.join(img_training, f"{i}.png")
    mask_path = os.path.join(img_training, f"{i}_mask.png")

    # Solo agrega las rutas si tanto la imagen como la máscara son válidas
    if os.path.exists(rgb_path) and os.path.exists(mask_path) and is_image_valid(rgb_path) and is_image_valid(mask_path):
        row_imgs = {'z': [i], 'path': [rgb_path]}
        row_segs = {'z': [i], 'path': [mask_path]}

        df_imgs_train = pd.concat([df_imgs_train, pd.DataFrame(row_imgs)])
        df_segs_train = pd.concat([df_segs_train, pd.DataFrame(row_segs)])
    else:
        print(f"Omitiendo debido a imagen/máscara inválida o inexistente: {rgb_path}, {mask_path}")

print(df_imgs_train.tail())
print(df_segs_train.tail())

print('Created IDs for training images.')


100%|█████████████████████████████████████████████████████████████████████████████| 1998/1998 [00:02<00:00, 838.28it/s]

      z                                             path
0  1994  F:/Segmentacion_Automatica/new_dataset\1994.png
0  1995  F:/Segmentacion_Automatica/new_dataset\1995.png
0  1996  F:/Segmentacion_Automatica/new_dataset\1996.png
0  1997  F:/Segmentacion_Automatica/new_dataset\1997.png
0  1998  F:/Segmentacion_Automatica/new_dataset\1998.png
      z                                               path
0  1994  F:/Segmentacion_Automatica/new_dataset\1994_ma...
0  1995  F:/Segmentacion_Automatica/new_dataset\1995_ma...
0  1996  F:/Segmentacion_Automatica/new_dataset\1996_ma...
0  1997  F:/Segmentacion_Automatica/new_dataset\1997_ma...
0  1998  F:/Segmentacion_Automatica/new_dataset\1998_ma...
Created IDs for training images.


In [14]:
from tensorflow.keras.models import load_model

'''Training'''
# Set random seed for reproducible training
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
rand=np.random.seed(SEED)
random.seed(SEED)
tf.compat.v1.set_random_seed(SEED)

# Set parameters
BATCH_SIZE = 8
FILTERS = 16
LAYERS= 4
LEARNING_RATE = 0.2e-3
EPOCHS = 30
TARGET_HEIGHT = 512
TARGET_WIDTH = 256
SHUFFLE = True
AUGMENT = False
MODEL_PATH = r"F:\Segmentacion_Automatica\Model\steps\epoch070.h5" #TODO
STEPS_PATH = r"F:\Segmentacion_Automatica\rt_model\steps" #TODO
N_STEPS = 5 # Save every #N_STEPS epoch
RADIUS = 15

model = load_model(MODEL_PATH, custom_objects={'dice_loss': dice_loss, 'iou_score': iou_score, 'mape_ap': mape_ap, 'mape_pp': mape_pp, 'metric_mape': metric_mape})

# Hard split of training and validation data
train_imgs, val_imgs, train_segs, val_segs = train_test_split(df_imgs_train,
                                                              df_segs_train,
                                                              test_size = 0.1,
                                                              random_state = SEED)


# Training data --> Augmentation and Shuffle
#training_generator = DataGenerator(train_imgs, train_segs, batch_size = BATCH_SIZE, target_height = TARGET_HEIGHT, \
                                   #target_width = TARGET_WIDTH, shuffle = SHUFFLE, df_coordinates = training_data, \
                                   #augment = AUGMENT, radius=RADIUS)
training_generator = DataGenerator(train_imgs, train_segs, batch_size = BATCH_SIZE, target_height = TARGET_HEIGHT, \
                                   target_width = TARGET_WIDTH, shuffle = SHUFFLE, augment = AUGMENT, radius=RADIUS)
# Validation data
validation_generator = DataGenerator(val_imgs, val_segs, target_height = TARGET_HEIGHT, \
                                     target_width = TARGET_WIDTH, batch_size = BATCH_SIZE, shuffle = False, \
                                    augment = False, radius=RADIUS)
#validation_generator = DataGenerator(val_imgs, val_segs, target_height = TARGET_HEIGHT, \
                                     #target_width = TARGET_WIDTH, batch_size = BATCH_SIZE, shuffle = False, \
                                     #df_coordinates = training_data, augment = False, radius=RADIUS)
# Compile model with dice_loss for segmentation, mse for prediction maps and use Adam as optimizer
# First exit: predictions of anterior and posterior points (2 channels)
# Second exit: Segmentations

for layer in model.layers:
    if 'decode_ap' in layer.name or 'ap.pred' in layer.name:
        layer.trainable = False

# Puedes verificar que las capas están congeladas imprimiendo el estado de cada capa
for layer in model.layers:
    print(layer.name, layer.trainable)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              metrics={'seg': ['acc', iou_score]},
              loss={'seg': dice_loss})


# Train model on dataset and save it
model.fit(training_generator, validation_data= validation_generator, epochs = EPOCHS,
                    callbacks = get_callbacks(MODEL_PATH, model, N_STEPS, STEPS_PATH))


input_2 True
conv2d_9 True
instance_normalization_27 True
activation_27 True
conv2d_10 True
instance_normalization_28 True
activation_28 True
max_pooling2d_4 True
conv2d_11 True
instance_normalization_29 True
activation_29 True
conv2d_12 True
instance_normalization_30 True
activation_30 True
max_pooling2d_5 True
conv2d_13 True
instance_normalization_31 True
activation_31 True
conv2d_14 True
instance_normalization_32 True
activation_32 True
max_pooling2d_6 True
conv2d_15 True
instance_normalization_33 True
activation_33 True
conv2d_16 True
instance_normalization_34 True
activation_34 True
max_pooling2d_7 True
conv2d_17 True
instance_normalization_35 True
activation_35 True
decode_apconv10 False
decode_segconv10 True
instance_normalization_45 True
instance_normalization_36 True
activation_45 True
activation_36 True
decode_apUpSampling0 False
decode_segUpSampling0 True
concatenate_12 True
concatenate_8 True
decode_apconv20 False
decode_segconv20 True
instance_normalization_46 True
instanc

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (8,) + inhomogeneous part.

In [11]:
from PIL import Image
import os

# Directorio donde se encuentran tus imágenes
img_directory = "F:/Segmentacion_Automatica/new_dataset"

# Lista para guardar las rutas de las imágenes que no se pueden abrir
corrupted_images = []

# Recorre las imágenes en el directorio
for filename in os.listdir(img_directory):
    if filename.endswith(".png") or filename.endswith(".jpg"): # Ajusta las extensiones según sea necesario
        img_path = os.path.join(img_directory, filename)
        try:
            with Image.open(img_path) as img:
                img.verify()  # Intenta abrir la imagen para verificar si está dañada
        except (IOError, SyntaxError) as e:
            print(f"Imagen dañada: {img_path} - {e}")
            corrupted_images.append(img_path)

# Imprime las imágenes dañadas, si las hay
if corrupted_images:
    print("Se encontraron imágenes dañadas:")
    for path in corrupted_images:
        print(path)
else:
    print("No se encontraron imágenes dañadas.")


No se encontraron imágenes dañadas.


In [12]:
import pandas as pd
import os
from tqdm import tqdm

# Asumiendo que ya has creado df_imgs_train y df_segs_train

# 1. Verificar las primeras y últimas filas de cada DataFrame
print("Primeras filas de df_imgs_train:")
print(df_imgs_train.head())
print("\nÚltimas filas de df_imgs_train:")
print(df_imgs_train.tail())

print("\nPrimeras filas de df_segs_train:")
print(df_segs_train.head())
print("\nÚltimas filas de df_segs_train:")
print(df_segs_train.tail())

# 2. Verificar el número total de filas
print(f"\nNúmero total de imágenes en df_imgs_train: {len(df_imgs_train)}")
print(f"Número total de máscaras en df_segs_train: {len(df_segs_train)}")

# 3. Verificar la existencia de las rutas de archivo
missing_files = {'img': [], 'seg': []}
for _, row in tqdm(df_imgs_train.iterrows(), total=df_imgs_train.shape[0], desc="Verificando imágenes"):
    if not os.path.exists(row['path']):
        missing_files['img'].append(row['path'])

for _, row in tqdm(df_segs_train.iterrows(), total=df_segs_train.shape[0], desc="Verificando máscaras"):
    if not os.path.exists(row['path']):
        missing_files['seg'].append(row['path'])

if not missing_files['img'] and not missing_files['seg']:
    print("\nTodas las rutas de archivo existen.")
else:
    print("\nArchivos faltantes:")
    print("Imágenes:", missing_files['img'])
    print("Máscaras:", missing_files['seg'])

# 4. Comprobar la consistencia de las rutas
# Esto dependerá de tu estructura de datos específica, pero puedes añadir aquí comprobaciones adicionales si es necesario.


Primeras filas de df_imgs_train:
   z                                          path
0  1  F:/Segmentacion_Automatica/new_dataset\1.png
0  2  F:/Segmentacion_Automatica/new_dataset\2.png
0  3  F:/Segmentacion_Automatica/new_dataset\3.png
0  4  F:/Segmentacion_Automatica/new_dataset\4.png
0  5  F:/Segmentacion_Automatica/new_dataset\5.png

Últimas filas de df_imgs_train:
      z                                             path
0  1994  F:/Segmentacion_Automatica/new_dataset\1994.png
0  1995  F:/Segmentacion_Automatica/new_dataset\1995.png
0  1996  F:/Segmentacion_Automatica/new_dataset\1996.png
0  1997  F:/Segmentacion_Automatica/new_dataset\1997.png
0  1998  F:/Segmentacion_Automatica/new_dataset\1998.png

Primeras filas de df_segs_train:
   z                                               path
0  1  F:/Segmentacion_Automatica/new_dataset\1_mask.png
0  2  F:/Segmentacion_Automatica/new_dataset\2_mask.png
0  3  F:/Segmentacion_Automatica/new_dataset\3_mask.png
0  4  F:/Segmentacion_Automa

Verificando máscaras: 100%|██████████████████████████████████████████████████████| 1998/1998 [00:00<00:00, 6977.28it/s]


Todas las rutas de archivo existen.


In [13]:
import os
from PIL import Image

# Directorio donde se encuentran tus imágenes y máscaras
img_directory = "/content/drive/MyDrive/Glottis/re_dataset/new_dataset"

# Lista para guardar las rutas de las imágenes/máscaras con dimensiones inconsistentes
inconsistent_images = []

# Recorre las imágenes en el directorio
for i in range(1, 2000):
    img_path = os.path.join(img_directory, f"{i}.png")
    mask_path = os.path.join(img_directory, f"{i}_mask.png")

    # Comprueba si ambos archivos existen
    if os.path.exists(img_path) and os.path.exists(mask_path):
        img = Image.open(img_path)
        mask = Image.open(mask_path)
        # Compara las dimensiones de la imagen y la máscara
        if img.size != mask.size:
            # Agrega a la lista y imprime el mensaje específico
            inconsistent_images.append((img_path, mask_path))
            print(f"Dimensiones inconsistentes: Imagen {img_path} [{img.size}], Máscara {mask_path} [{mask.size}]")

# Si no hay imágenes inconsistentes, imprime un mensaje
if not inconsistent_images:
    print("Todas las imágenes y máscaras tienen dimensiones consistentes.")

Todas las imágenes y máscaras tienen dimensiones consistentes.
